# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD model for fitting the actual Chilean data in order to make projections in VMI and bed usage, amount of infected and deaths. This model uses the SEIRHVD 4.1 version. A visual representation of the model is available at: [SEIRHVD Miró Model](https://miro.com/app/board/o9J_ktzN4xA=/)

This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from datetime import datetime
from numpy import linalg as LA
import multiprocessing
from joblib import Parallel, delayed
from datetime import timedelta

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../../src/SEIRHVD/')
sys.path.insert(1, '../../src/utils/')
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')

from class_SEIRHUVD5 import SEIRHVD 
from Quarantine import Quarantine
from Quarantine import SeroPrevDynamics
from importdata import ImportData

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [2]:
# First simulation Initial date
initdate = datetime(2020,9,1)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days


In [3]:
# holydays
holiday = [[datetime(2020,11,26),'Thanksgiving'],
           [datetime(2020,11,3),'Presidential Election'],
           [datetime(2020,10,31),'Halloween'],
           [datetime(2020,12,25),'Christmas']]
           #[datetime(2020,10,31),'Trump Rallies']
          #]

thanksgiving = datetime(2020,11,26)
presidentialelection = datetime(2020,11,3)
halloween = datetime(2020,10,31)

## Import Data

In [4]:
# NY State population
population = 19450000

In [5]:
datadir = '/home/samuel/Dropbox/DLab/Data/USA/'
data = pd.read_csv(datadir+'ny_state_NYT.csv')
hospdata = pd.read_csv(datadir+'ny_state_BEDS.csv')

# Infected
I_ac_r = data['cases']
I_d_r = data['cases'].diff()
dates = [datetime.strptime(data['date'][i],'%Y-%m-%d') for i in range(len(data))]

# Deaths
D_ac_r = data['deaths']
D_d_r = data['deaths'].diff()


# hospitals
hosp_beds_cap = hospdata['inpatient_beds']
hosp_beds_gen = hospdata['inpatient_beds_used']
hosp_beds_covid = hospdata['inpatient_beds_used_covid']

UCI_cap = hospdata['total_staffed_adult_icu_beds']
UCI_gen = hospdata['staffed_adult_icu_bed_occupancy']
UCI_covid = hospdata['staffed_icu_adult_patients_confirmed_covid']

hosp_dates = [datetime.strptime(hospdata['date'][i],'%Y-%m-%d') for i in range(len(dates))]


FileNotFoundError: [Errno 2] No such file or directory: '/home/samuel/Dropbox/DLab/Data/USA/ny_state_NYT.csv'

### Data plots

In [6]:
plt.plot(dates,I_ac_r,label='Accumulated Infections')
plt.plot(dates,D_ac_r,label='Accumulated Deaths')
plt.legend(loc=0)
plt.title('Ny State - EPI accumulated')
plt.show()

In [7]:
plt.plot(dates,I_d_r,label='Daily Infections')
plt.plot(dates,D_d_r,label='Daily Deaths')
plt.legend(loc=0)
plt.title('Ny State - EPI daily')
plt.show()

In [242]:
plt.plot(dates,np.log10(I_d_r),label='Daily Infections')
#plt.plot(dates,D_d_r,label='Daily Deaths')
plt.legend(loc=0)
plt.title('Ny State - EPI daily')
plt.show()

In [977]:
plt.plot(hosp_dates,hosp_beds_cap,label='Hospital capacity',color = 'red',linestyle = 'dashed')
plt.plot(hosp_dates,hosp_beds_gen,label='Hospital use total')
plt.plot(hosp_dates,hosp_beds_covid,label='Hospital use covid')

plt.legend(loc=0)
plt.title('Ny State - Hospitals general beds')
plt.show()

In [979]:
plt.plot(hosp_dates,UCI_cap,label='UCI capacity',color = 'red',linestyle = 'dashed')
plt.plot(hosp_dates,UCI_gen,label='UCI use total')
plt.plot(hosp_dates,UCI_covid,label='UCI use covid')

plt.legend(loc=0)
plt.title('Ny State - ICU')
plt.show()

## Initial values
Los acumulados de 2 semanas antes del inicio de la simulacion

In [907]:
rday  = (initdate - timedelta(days=14)-dates[0]).days
R0 = I_ac_r[rday] - D_ac_r[rday]

data_initday = (initdate-dates[0]).days
hospdata_initday = (initdate-hosp_dates[0]).days

Htot0 =  hosp_beds_cap[hospdata_initday] + 10000
Vtot0 =  UCI_cap[hospdata_initday]+ 10000

H0 = hosp_beds_covid[hospdata_initday]
V0 = UCI_covid[hospdata_initday] 

B0 = D_ac_r[data_initday]
D0 = D_d_r[data_initday]


I0 = I_ac_r[data_initday] - I_ac_r[rday]
I_d0 = I_d_r[data_initday]
I_ac0 = I_ac_r[data_initday]

## Simulation Parameters

In [908]:
# Total simulation time
tsim = 365

beta = 0.088
mu = 0.03
k_I = 0
k_R = 0.1
SeroPrevFactor = 0.1
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [909]:
# Quarantine relaxation
#SPchange_date = datetime(2020,9,11)
#t_sp = (SPchange_date - initdate).days

# Quarantines 
max_mob = 0.6
rem_mob = 0.48

fqtdays = (datetime(2020,11,1)-initdate).days

Q1 = Quarantine(rem_mob,max_mob,qp=0,iqt=0,fqt=fqtdays)
Q2 = Quarantine(rem_mob)

alpha = Q1.alpha

## Underreport
Fraction of Infected detected/reported  

$Imi_{det}$: Fraction of Mild detected   
$Ias_{det}$: fraction of asymptomatic detected  

If both are 1, means that all infected are detected so 
\begin{align}
I_{det} = I
\end{align}

In [910]:
Imi_det = 1
Ias_det = 1
Ise_det = 1
Icr_det = 1

## State Parameters:
The following parameters determine transitions probability and duration between different states. The values presented here are the default values set in the model. You can change them as it's showed in the following executable lines.

pE_Ias = 0.4  # Transition from exposed to Asymptomatic Infected  
tE_Ias = 5.0

pE_Imi = 0.55 # Transition from exposed to  Mild Infected  
tE_Imi = 5.0

pE_Icr = 0.01666 # Transition from exposed to  Critical Infected  
tE_Icr = 3.0

pE_Ise = 0.03334 # Transition from exposed to  Serious Infected  
tE_Ise = 3.0

pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered  
tIas_R = 10.0

pImi_R = 1.0  # Transition from Mild Infected to Recovered  
tImi_R = 15.0

pIse_Hse = 1.0 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)  
tIse_Hse = 3.0 

pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)  
tIse_D = 3.0         

pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)  
tIcr_V = 3.0 

pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)  
tIcr_D = 3.0         

pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered  
tHse_R = 11.0

pHse_V = 0.03 # Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)  
tHse_V = 3.0

pHse_D = 0.03 # Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)  
tHse_D = 3.0        

pV_Hout = 0.5  # Transition from Ventilators to Hospital Recovery (Hout)   
tV_Hout = 15.0

pV_D = 0.5 # Transition from Ventilators to Death  
tV_D = 15.0

pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered  
tHout_R = 4.0

pD_B = 1.0 # Transition from Dead to buried  
tD_B = 1.0 

betaD = 0 # Contagion by deads rate  
eta = 0.0 # Immunity loss rate  

In [911]:
#state parameters: 
pE_Ias = 0.3 # Transition from exposed to Asymptomatic Infected
tE_Ias = 4.0
pE_Imi = 0.43 # Transition from exposed to  Mild Infected
tE_Imi = 4.0
pE_Icr = 0.006 # Transition from exposed to  Critical Infected
tE_Icr = 3.0
pE_Ise = 0.254# Transition from exposed to  Serious Infected
tE_Ise = 3.0
pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered
tIas_R = 9.0
pImi_R = 1.0  # Transition from Mild Infected to Recovered
tImi_R = 12.0
pIse_Hse = 1 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)
tIse_Hse = 5.0
pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)
tIse_D = 3.0
pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)
tIcr_V = 3.0
pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)
tIcr_D = 3.0
pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered
tHse_R = 3.5
pHse_V = 0.03# Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)
tHse_V = 4.0
pHse_D = 0.85# Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)
tHse_D = 4.0
pV_Hout = 0.34  # Transition from Ventilators to Hospital Recovery (Hout)
tV_Hout = 16.0
pV_D = 0.66 # Transition from Ventilators to Death
tV_D = 12.0
pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered
tHout_R = 10.0

## SeroPrevalence Dynamics
The susceptibles increase through a step function which last for the specified amount of days. Throughout these days it adds a "dailyincrease" amount of persons each day.

The total increase of people is increasedays*dailyincrease

In [912]:
spdate = datetime(2020,10,10)
t_sp = (spdate - initdate).days

increasedays = 30
renewalFactor = 0.4 #0.23
dailyincrease = population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = t_sp + 0
chi0 = SeroPrevDynamics(t_sp_temp+5,t_sp_temp+increasedays*0.7,t_sp_temp+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)
chi = [chi0]
#renewalFactor = [renewalFactor]

In [913]:
def chisum(a,b):
    def aux(t):
        return a(t)+b(t)
    return aux

In [914]:
renewalFactor = [-0.2,-0.1,0,0.1,0.2,0.3]

In [915]:
# Increase date 
increasedate = datetime(2020,12,15)
increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 30
# Daily amount of people
dailyincrease = [population*SeroPrevFactor*i/increasedays for i in renewalFactor]

In [916]:
chi = [chisum(chi0,SeroPrevDynamics(increaseday,increaseday+increasedays*0.0001,increaseday+increasedays,i)) for i in dailyincrease]

In [917]:
#taux = np.array(range(180))
#for i in range(len(chi)):
#    plt.plot(taux,chi[i](taux))
#plt.show()

# Simulation

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

In [918]:
simulation = [SEIRHVD(tsim,beta,mu,alpha,k_I=k_I,k_R = k_R, chi = i, 
                     SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,Htot=int(Htot0),
                     Vtot=int(Vtot0),H0=H0,V0=V0,B0=B0,D0=D0,R0=R0,I0=I0,I_d0=I_d0,I_ac0=I_ac0,
                     population=population,Imi_det = Imi_det,Ias_det = Ias_det,initdate=initdate) for i in chi]

Modify simulation state parameters

In [919]:
for i in range(len(simulation)):
    simulation[i].pE_Ias=pE_Ias
    simulation[i].tE_Ias=tE_Ias
    simulation[i].pE_Imi=pE_Imi
    simulation[i].tE_Imi=tE_Imi
    simulation[i].pE_Icr=pE_Icr
    simulation[i].tE_Icr=tE_Icr
    simulation[i].pE_Ise=pE_Ise
    simulation[i].tE_Ise=tE_Ise
    simulation[i].pIas_R=pIas_R
    simulation[i].tIas_R =tIas_R
    simulation[i].pImi_R=pImi_R
    simulation[i].tImi_R =tImi_R
    simulation[i].pIse_Hse=pIse_Hse
    simulation[i].tIse_Hse=tIse_Hse
    simulation[i].pIse_D=pIse_D
    simulation[i].tIse_D=tIse_D
    simulation[i].pIcr_V=pIcr_V
    simulation[i].tIcr_V=tIcr_V
    simulation[i].pIcr_D=pIcr_D
    simulation[i].tIcr_D=tIcr_D
    simulation[i].pHse_R=pHse_R
    simulation[i].tHse_R =tHse_R
    simulation[i].pHse_V=pHse_V
    simulation[i].tHse_V=tHse_V
    simulation[i].pHse_D=pHse_D
    simulation[i].tHse_D=tHse_D
    simulation[i].pV_Hout=pV_Hout
    simulation[i].tV_Hout =tV_Hout
    simulation[i].pV_D=pV_D
    simulation[i].tV_D =tV_D
    simulation[i].pHout_R=pHout_R
    simulation[i].tHout_R=tHout_R
    simulation[i].setnewparams()

Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed


### Setting new params

## Run simulation

* **integr_sci**: Fast solver, sometimes stiffness beat it  
* **integr:** Slow solver, more robust with stiffness  

In [920]:
def simulate(simulation,i,tsim):
    simulation[i].integr_sci(0,tsim,0.1)
    return simulation[i]

In [921]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(simulation,i,tsim) for i in range(len(simulation)))
simulation = sims
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=12)]: Done   2 out of   6 | elapsed:    1.5s remaining:    3.0s
[Parallel(n_jobs=12)]: Done   3 out of   6 | elapsed:    1.6s remaining:    1.6s
[Parallel(n_jobs=12)]: Done   4 out of   6 | elapsed:    1.6s remaining:    0.8s
[Parallel(n_jobs=12)]: Done   6 out of   6 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   6 out of   6 | elapsed:    1.7s finished
ready


def simulate(simulation,i,tsim):
    simulation[i].integr_sci(0,tsim,0.1)
    return simulation[i]

num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(simulation,i,tsim) for i in range(len(simulation)))
simulation = sims
print('ready')

# Analysis

## Grid Plot

In [922]:
# Days to plot
days = 300
enddate = initdate + timedelta(days=2*365)
enddate = datetime(2021,2,28)

In [971]:
saveplot = False
if saveplot:
    %matplotlib inline
else:
    %matplotlib tk
#%matplotlib tk
plt.rcParams["figure.figsize"] = 38.40, 20.56 #8,6
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif":["Arial"]})


### Daily info

In [975]:
colors = plt.cm.rainbow(np.linspace(0,1,len(holiday)))


fig, axs = plt.subplots(2, 2)

# --------------------------------------- #
#          New Daily Infections           #
# --------------------------------------- #


#axs[0,0].plot([], [], ' ', label='beta = '+str(beta))
#axs[0,0].plot([], [], ' ', label='mu = '+str(mu))
#axs[0,0].plot([], [], ' ', label='seroprev = '+str(SeroPrevFactor))

axs[0,0].scatter(dates[data_initday:],I_d_r[data_initday:],label='Real Data')
for i in range(len(simulation)):
    axs[0,0].plot(simulation[i].dates,simulation[i].I_d,label='RF: '+str(renewalFactor[i]))

# plot holidays:
for i in range(len(holiday)):
    axs[0,0].axvline(x = holiday[i][0], linestyle = 'dashed',color = colors[i],label=holiday[i][1])

axs[0,0].axvline(x = datetime(2021,1,28), linestyle = 'dashed',color = 'grey',label='Peak period start')
axs[0,0].axvline(x = datetime(2021,2,10), linestyle = 'dashed',color = 'grey',label='Peak period end')

axs[0,0].set_title('New Infections',size=25)
axs[0,0].set_xlim(initdate+timedelta(days=14),enddate)
axs[0,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[0,0].tick_params(labelsize=20)
axs[0,0].set_xlabel('Date',size=25)
axs[0,0].set_ylabel('New Infections',size=25)
axs[0,0].legend(loc=0,fontsize='16',title_fontsize='25')



# --------------------------------------- #
#            New Daily Deaths             #
# --------------------------------------- #

axs[0,1].scatter(dates[data_initday:],D_d_r[data_initday:],label='Real Data')

for i in range(len(simulation)):
    axs[0,1].plot(simulation[i].dates,simulation[i].D,label='RF: '+str(renewalFactor[i]))
    
    
# plot holidays:
for i in range(len(holiday)):
    axs[0,1].axvline(x = holiday[i][0], linestyle = 'dashed',color = colors[i],label=holiday[i][1])
    
axs[0,1].set_title('New Deaths',size=25)
axs[0,1].set_xlim(initdate+timedelta(days=14),enddate)
axs[0,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[0,1].tick_params(labelsize=20)
axs[0,1].set_xlabel('Date',size=25)
axs[0,1].set_ylabel('New Deaths',size=25)
axs[0,1].legend(loc=0,fontsize='16',title_fontsize='25')

# --------------------------------------- #
#          Normal Hospital beds           #
# --------------------------------------- #

axs[1,0].scatter(hosp_dates[hospdata_initday:],hosp_beds_covid[hospdata_initday:],label='Real Data')

for i in range(len(simulation)):
    axs[1,0].plot(simulation[i].dates,simulation[i].H,label='RF: '+str(renewalFactor[i]))
    
# plot holidays:
for i in range(len(holiday)):
    axs[1,0].axvline(x = holiday[i][0], linestyle = 'dashed',color = colors[i],label=holiday[i][1])

axs[1,0].set_title('Hospital beds use',size=25)
axs[1,0].set_xlim(initdate+timedelta(days=14),enddate)
axs[1,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[1,0].tick_params(labelsize=20)
axs[1,0].set_xlabel('Date',size=25)
axs[1,0].set_ylabel('Hospital beds use',size=25)
axs[1,0].legend(loc=0,fontsize='16',title_fontsize='25')


# --------------------------------------- #
#                  ICU                   #
# --------------------------------------- #


axs[1,1].scatter(hosp_dates[hospdata_initday:],UCI_covid[hospdata_initday:],label='Real Data')

for i in range(len(simulation)):
    axs[1,1].plot(simulation[i].dates,simulation[i].V,label='RF: '+str(renewalFactor[i]))
    

# plot holidays:
for i in range(len(holiday)):
    axs[1,1].axvline(x = holiday[i][0], linestyle = 'dashed',color = colors[i],label=holiday[i][1])


axs[1,1].set_title('ICU use',size=25)
axs[1,1].set_xlim(initdate+timedelta(days=14),enddate)
axs[1,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[1,1].tick_params(labelsize=20)
axs[1,1].set_xlabel('Date',size=25)
axs[1,1].set_ylabel('ICU use',size=25)
axs[1,1].legend(loc=0,fontsize='16',title_fontsize='25')


fig.suptitle('NewYork State',fontsize=30)

if saveplot:
    fig.savefig('/home/samuel/Dropbox/DLab/Data/USA/NYForecast_Dec2020.pdf',dpi=100,format='pdf')

### Accumulated info

In [904]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(holiday)))

fig, axs = plt.subplots(2, 2)

# --------------------------------------- #
#          Accumulated Infected           #
# --------------------------------------- #
axs[0,0].set_title('Accumulated Infected')
axs[0,0].set_xlim(initdate,enddate)
axs[0,0].plot([], [], ' ', label='beta = '+str(beta))
axs[0,0].plot([], [], ' ', label='mu = '+str(mu))
axs[0,0].plot([], [], ' ', label='seroprev = '+str(SeroPrevFactor))

axs[0,0].scatter(dates[data_initday:],I_ac_r[data_initday:],label='Real')
axs[0,0].plot(sim.dates,sim.I_ac,label='Simulation')

# plot holidays:
for i in range(len(holiday)):
    axs[0,0].axvline(x = holiday[i][0], linestyle = 'dotted',color = colors[i],label=holiday[i][1])


axs[0,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[0,0].legend(loc=0)



# --------------------------------------- #
#          Accumulated Deaths             #
# --------------------------------------- #


axs[0,1].set_title('Accumulated Deaths')
axs[0,1].set_xlim(initdate,enddate)

axs[0,1].scatter(dates[data_initday:],D_ac_r[data_initday:],label='Real')
axs[0,1].plot(sim.dates,sim.B,label='Simulation')

# plot holidays:
for i in range(len(holiday)):
    axs[0,1].axvline(x = holiday[i][0], linestyle = 'dotted',color = colors[i],label=holiday[i][1])
    
axs[0,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[0,1].legend(loc=0)


# --------------------------------------- #
#          Normal Hospital beds           #
# --------------------------------------- #

axs[1,0].set_title('Hospital beds use')
axs[1,0].set_xlim(initdate,enddate)

axs[1,0].scatter(hosp_dates[hospdata_initday:],hosp_beds_covid[hospdata_initday:],label='Real')
axs[1,0].plot(sim.dates,sim.H,label='Simulation')

# plot holidays:
# plot holidays:
for i in range(len(holiday)):
    axs[1,0].axvline(x = holiday[i][0], linestyle = 'dotted',color = colors[i],label=holiday[i][1])

axs[1,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[1,0].legend(loc=0)


# --------------------------------------- #
#                  UCI                    #
# --------------------------------------- #

axs[1,1].set_title('UCI use')
axs[1,1].set_xlim(initdate,enddate)

axs[1,1].scatter(hosp_dates[hospdata_initday:],UCI_covid[hospdata_initday:],label='Real')
axs[1,1].plot(sim.dates,sim.V,label='Simulation')

# plot holidays:
# plot holidays:
for i in range(len(holiday)):
    axs[1,1].axvline(x = holiday[i][0], linestyle = 'dotted',color = colors[i],label=holiday[i][1])
    
axs[1,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[1,1].legend(loc=0)


fig.suptitle('NewYork State',fontsize=16)


Text(0.5, 0.98, 'NewYork State')

## Single plot

In [905]:
sim = simulation[0]

In [747]:
# Accumulated Infected
plt.scatter(dates[data_initday:],I_ac_r[data_initday:],label='Real')
plt.plot(sim.dates,sim.I_ac,label='Simulation')
plt.title('Accumulated Infected')

Text(0.5, 1.0, 'Accumulated Infected')

In [748]:
# Daily Infected
plt.scatter(dates[data_initday:],I_d_r[data_initday:],label='Real')
plt.plot(sim.dates,sim.I_d,label='Simulation')
plt.title('New Daily Infected')

Text(0.5, 1.0, 'New Daily Infected')

In [25]:
# Accumulated Deaths
plt.scatter(dates[data_initday:],D_ac_r[data_initday:],label='Real')
plt.plot(sim.dates,sim.B,label='Simulation')
plt.title('Accumulated Deaths')

Text(0.5, 1.0, 'Accumulated Deaths')

In [26]:
# New Daily Deaths
plt.scatter(dates[data_initday:],D_d_r[data_initday:],label='Real')
plt.plot(sim.dates,sim.D,label='Simulation')
plt.title('New Daily Deaths')

Text(0.5, 1.0, 'New Daily Deaths')

In [27]:
# Hospital Use
plt.scatter(hosp_dates[hospdata_initday:],hosp_beds_covid[hospdata_initday:],label='Real')
plt.plot(sim.dates,sim.H,label='Simulation')
plt.title('General beds Use')

Text(0.5, 1.0, 'General beds Use')

In [30]:
# UCI Use
plt.scatter(hosp_dates[hospdata_initday:],UCI_covid[hospdata_initday:],label='Real')
plt.plot(sim.dates,sim.V,label='Simulation')
plt.title('UCI Use')

Text(0.5, 1.0, 'UCI Use')

In [31]:
SeroPrevalence = [simulation[i].I_ac[-1]/simulation[i].population for i in range(len(simulation))]
SeroPrevalenceDet = [simulation[i].I_ac_det[-1]/simulation[i].population for i in range(len(simulation))]